In [1]:
import cv2
import time
from IPython.display import display, clear_output
import mediapipe as mp
from PIL import Image

# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils  # Initialize drawing utilities

# Open a connection to the webcam
cap = cv2.VideoCapture(0)

# Track time when no face is detected
no_face_start = None
face_detected = False

# Function to display the frame in Jupyter Notebook
def display_frame(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img_pil = Image.fromarray(img_rgb)  # Convert to PIL Image
    display(img_pil)  # Display the image
    clear_output(wait=True)  # Clear previous output for smooth updates

try:
    while cap.isOpened():
        success, frame = cap.read()  # Read a frame from the webcam
        if not success:
            print("Ignoring empty camera frame.")
            break

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        # Check if faces are detected
        if len(faces) > 0:
            face_detected = True
            no_face_start = None  # Reset the no-face timer

            # Draw rectangles around the faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        else:
            if face_detected:
                face_detected = False
                no_face_start = time.time()  # Start the no-face timer

        # Convert the frame to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Process the frame and find hands
        result = hands.process(frame_rgb)

        # Draw hand landmarks if any hands are detected
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the frame
        display_frame(frame)

        # Check if no face has been detected for 5 seconds
        if no_face_start and (time.time() - no_face_start) > 5:
            print("No face detected for 5 seconds. Exiting...")
            break

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Release the webcam and clean up
    cap.release()
    cv2.destroyAllWindows()

No face detected for 5 seconds. Exiting...
